In [1]:
using DelimitedFiles # For writing the population out
using Distributions  # For SkewNormal, etc
using StatsBase	     # For the weighted sampling
using Plots
using StatsPlots, KernelDensity
using Distributed
using PlotThemes
theme(:dark)
addprocs()

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [2]:
@everywhere using DelimitedFiles # For writing the population out
@everywhere using Distributions  # For SkewNormal, etc
@everywhere using StatsBase	     # For the weighted sampling
@everywhere using Plots
@everywhere using StatsPlots, KernelDensity
@everywhere using Distributed
@everywhere using SharedArrays
@everywhere using DistributedArrays

In [3]:
@sync @distributed for kk in 1:nworkers()
numruns = 100;
numdims = 2;
numind = 2;
ac = 0.03;
maxtt = 1000000;   # maximum number of generations to run
var = 0.01;   # Variance of multivariate normal diffusion kernel
# global ac = 0.3;   # Euclidian distance defining the boundaries of "arbitrarily close"
for ll in 1:5
ns = (ll/10)
x = zeros(numdims,numind);
x0 = zeros(numdims,numind);
m = zeros(numdims);
cv = zeros(numdims,numdims);
for i in 1:numdims
    m[i] = 0;
end
for i in 1:numdims
    for j in 1:numdims
        if i == j
            cv[i,j] = var;
        end
    end
end
ip = 0.7;
for i in 1:numdims   # note: this only works with two starting individuals!
    x0[i,1] = ip;
    x0[i,2] = -ip;
end
    
t1=[];
for _ in 1:numruns
tt = 0;
x = deepcopy(x0);
global mpd = 0;
while (mpd < 1) && (tt < maxtt)
    tt += 1;
    for i in 1:numind
        x[:,i] += rand(MvNormal(m,cv));
    end
    mpd = 0;   #minimum pairwise distance
    for i in 1:numind-1
        for j in i+1:numind
            d1 = sum([ (x[k,i] - x[k,j])^2 for k in 1:numdims ]);
            pr1 = (π*ns^2)*exp(-d1/(2*ns^2));
            if rand() < pr1
                mpd = 1;
            end
        end
    end
end
if (tt > 1) && (tt < maxtt)
    push!(t1,tt);
end

end
# histogram(t1)
    
    fn0 = "out4/2Dn3-" * string(kk) *"-"* string(ll) * ".csv";
    writedlm( fn0,  t1, ',');
    
end

end



Task (done) @0x00002ba1bb92b5b0

In [ ]:
π

In [9]:
writedlm( "t1e.csv",  t1, ',')